# FFT

In [1]:
from spectrochempy import *

"<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABgAAAAYCAYAAADgdz34AAAAAXNSR0IArs4c6QAAAAlw SFlzAAAJOgAACToB8GSSSgAAAetpVFh0WE1MOmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6 bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDUuNC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAgICAgICAg eG1sbnM6eG1wPSJodHRwOi8vbnMuYWRvYmUuY29tL3hhcC8xLjAvIgogICAgICAgICAgICB4bWxuczp0aWZmPSJodHRwOi8vbnMu YWRvYmUuY29tL3RpZmYvMS4wLyI+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+bWF0cGxvdGxpYiB2ZXJzaW9uIDIuMS4wLCBo dHRwOi8vbWF0cGxvdGxpYi5vcmcvPC94bXA6Q3JlYXRvclRvb2w+CiAgICAgICAgIDx0aWZmOk9yaWVudGF0aW9uPjE8L3RpZmY6 T3JpZW50YXRpb24+CiAgICAgIDwvcmRmOkRlc2NyaXB0aW9uPgogICA8L3JkZjpSREY+CjwveDp4bXBtZXRhPgqNQaNYAAAGiUlE QVRIDY1We4xU1Rn/3XPuYx47u8w+hnU38hTcuoUEt/6D2y4RB0ME1BoEd9taJaKh9CFiN7YGp7appUAMNmktMZFoJTYVLVQ0smsy 26CN0SU1QgsuFAaW3WVmx33N677O6XfuyoIxTXqSO/fec+75fd93vt/3/UbDV0aKSZmCpkFMLz3T9utuu2N+o98aDSMBKVAo89z5 y+zEz3ZafcCOfvWdlGCalqKn1Bf71CygTd+mf1esSOnpdMpTb+vWpTZuWVfe3jLPa5tzHYNm0T5N0gpdkkHaDBeGBU6d1/t/fyS8 +/CbqdfUvmsx1PuMgc2bNxv79u1zgd31r+7JH1jbIZKxWRXAcYUQ8IWvBfBXNjEuJWPgMA02NR7C3/pYT9fjdZ3A9tGrWF8YSJHn qcDz3y7q2T967PZv+gnYJdd1mEZ+62zGDQV/dQgKhmLzDNOXCEWM3j6eTT5Y3w78dOBKJLR1PQf+4ivPj76UPZnssBN+wbM9Aet/ AV81Mf1EEULXYfOobvX2WWQk0aoioXwwSmirOlioY0mu8BIouzYl7P8GV3vpqCCEZvlFz769w08oLDWvyKIyL1asSm28d6WfzA97 ztvvV1kexUMsmhlkULEkuGYmFYC6AvfUrITnwUKl5K79lkjeSSRRTCTbQPd95e1WzMbZSya74XoXAxctCllCnbECMOjZNGRwvzIX nD85wbkMmKK+U045Dtdi8Qp+SAxU2GTg2bYlC9224pgvmSb54vkVTBQYyhUt2KjAMyMmPjwRQW5Mh2WKwJhlBh6jVGagFM84wZnQ 4bpC0Rt4pk1PbSt0NDcxDA5xryosDHWgtbM0DGZDWLSoiDMDYeQnGVrmOThxLozB0RAaahzkJzjKNqcIQBymJFMkOlN8Dqjpg0XY Tx5xO/QbmmUrqIjGJznq47TqTaClKYfjp+PInLMwnOdYvtQBZ2XcunQY+VwIo4U4muoFEjVEFE6lQyEUKzHYfgQG9ylCyngU+Cxj tOqxCDGHcCsOMCs6iQul5ZiStdATYxjMZXDLTUVwLY8Jey4uOh2IxjwsrP8UXJYxUrkZrghBahzV5iXU6gNkq0Z1EzIsUBUSCV2n EOHo0LVxHCpuxabJJdhi5PFnvw5vLXwXIfNZvD/+JNo/X40NegE54sUaazl+UL8XD1x+FB9Ijjt4EQfdGN6J/x131LwIV9ap/AYs 0x1fz1ZKFbh6A7qKy/By9Dg6G36Ep91vUJJ15Cqr0Z67E8/HzmBrw1OwxWyM+3Mo6BAuSB17oyfx0Oyl2DN0Hqs/70Cx6hBCvESF UY1ShWXZZEE7OTAYxZzaPH4TuoiusZvRnunFy2NbiHYuBp2vB66srX4vMEjpRKPxKXmnoQ4+Mn4DPiv8CYcrs3GfNUXJLtM+alSO hrMj/KT+wBNW3+E/2liywNO3iSflbaFva/+stGDTxE0E9Sjaox8HBhxpEamzMGSEaFKg+mjEddzDh1MxTDq3YV1kGBsjfwW3S9Cq anjmko+ndlb1UR3s6K8JlfphNWq9Ew/7c61T2BB/EbcaNkb8GBaE0tANH7/M34PLdhJDzjIcL9xPbdTG6zyM72Y+wXPHmvB489No fm0b5HnbQ9Rgp/7DSSd29AeVvPeNyK6JcYl/yQVi5dBjuGvoV/gaJe47s45QUxrDmcYX0MBsdF7egvXZ7+O0vZA4X8QmOQWjlSK7 RDz5wIM30gp9UbWcGjXxhzdDu1SiNSpx6kcQB57rPnr/3dlkZarWLnlRq5oPET1dOCIOk4wALib9eeS5iygfhkd09H0DWphB/+gs +PcOAS+ssrFmmXXgVfR0de9cpbAJfH3Q1jofW9DZk56dDcVsq9YcsoUMEd1qyLoT3BX1YiyHMJuk97hyjqIoE91t+NcTLeN0ZrfM oXatZbu6G0h4VG+ibqq0IJVK6cAjo6serG3vSUezCMct0yQeSOFJSUImqb2qbknUpDqlZxE0QZ+ZUpSlZx79h4Nda6zef9dlk121 JDjbR5XggPRZlRnS6bRQRtLpn4++cuie/Yvn2svmNxuLw9WCcYIl4fEoTEGiSTUqJdfgU+8ROqf1iMkLzS389YtNPXc/PH8l8ONB JZkHD+4JtD04HmVEDWWErmBhzV2/2LB1bemJG6krzv2S6NOHUgtEP0Oif5pE/3fHoruP7N8RiP61GArzSwbUhJJQpXJKiKbfr/3b IhKq76sKPUdF9NW/LSqfSn6vjv8C45H/6FSgvZQAAAAASUVORK5CYII=' style='height:25px; border-radius:12px; display:inline-block; float:left; vertical-align:middle'>",SpectroChemPy's API - v.0.1a14.dev59+gab30d27.d20191215© Copyright 2014-2019 - A.Travert & C.Fernandez @ LCS


In [2]:
def _ps(data, p0=0.0, p1=0.0):
    """
    Linear phase correction

    Parameters
    ----------
    data : ndarray
        Array of NMR data.
    p0 : float
        Zero order phase in degree
    p1 : float
        First order phase in degree

    Returns
    -------
    out : ndarray
        Phased NMR data.

    """
    p0r = p0.copy().to('rad').m
    p1r = p1.copy().to('rad').m
    size = data.shape[-1]
    out = data * np.exp(1.0j * (p0r + p1r * np.arange(size)/ size)).astype(data.dtype)
    
    return out

## Import data

Here we import two dataset, one is 1D and the other is 2D

Because , we will sometimes need to recall the original dataset, we create two getting functions

In [3]:
# 1D dataset getting function
datadir = general_preferences.datadir
def get_dataset1D():
    dataset1D = NDDataset()
    path = os.path.join(datadir,'nmrdata','bruker', 'tests', 'nmr','bruker_1d')
    dataset1D.read_bruker_nmr(path, expno=1, remove_digital_filter=True)
    return dataset1D

# 2D dataset getting function
def get_dataset2D():
    dataset2D = NDDataset()
    path = os.path.join(datadir,'nmrdata','bruker', 'tests', 'nmr','bruker_2d')
    dataset2D.read_bruker_nmr(path, expno=1, remove_digital_filter=True)
    return dataset2D

In [4]:
dataset1D = get_dataset1D() # restore original
LB = 10.*ur.Hz
dataset1D.em(lb=LB)
transf1 = dataset1D.fft(size=32000)
_ = transf1.plot(xlim=[30,-30])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

by default, the frequency axis unit is in ppm when data originate from NMR. Note that the axis is automatically reversed as it is usual for ppm scale in NMR.

To get frequency scale, then set `ppm` flag to False

In [ ]:
from ipywidgets import interactive, fixed, FloatSlider
from IPython.display import display
#import matplotlib.pyplot as plt

def interact_pk(dataset, dim=-1, **kwargs):
    
    verbose = kwargs.get('verbose', False)
    if verbose:
        print_(' ')
        print_('INTERACTIVE PHASING MODE')
    
    new = dataset.copy()
    
    # On which axis do we want to phase? (get axis from arguments)
    # The last dimension is always the dimension on which we apply the phases.
    # If needed, we swap the dimensions to be sure to be in this situation
    axis, dim = new.get_axis(dim, negative_axis=True)
    swaped = False
    if axis != -1:
        new.swapaxes(axis, -1, inplace=True)  # must be done in  place
        swaped = True
    
    # select the last coordinates and check the unit validity
    lastcoord = new.coords[dim]
    if (lastcoord.units.dimensionality != '1/[time]' and lastcoord.units != 'ppm'):
        error_('Phasing apply only to dimensions with [frequency] dimensionality or with ppm units\n'
               'Phasing processing was thus cancelled')
        return new
    
    # get initial absolute phase and pivot
    # get the initial phase setting
    if verbose:
        print_(f'Current phases : {new.meta.phc0[-1]}, {new.meta.phc1[-1]}')
    
    phc0 = new.meta.phc0[-1]
    phc1 = new.meta.phc1[-1]
    pivot = float(abs(new).max().coords[-1].data) 
    ppivot = lastcoord.loc2index(pivot)
    
    ax = new.plot(xlim=[50,-50]).axes
    l = ax.lines[0]
    p = ax.axvline(pivot, color='r', alpha=0.5)
    
    def _phasing( ph0, ph1, pivot):
        
        rphc0 = (ph0 - phc0) * ur.deg
        rphc1 = (ph1 - phc1) * ur.deg
        ppivot = lastcoord.loc2index(pivot)
        
        data = _ps(new.data, rphc0, rphc1)
        
        l.set_ydata(data)
        p.set_xdata([pivot,pivot])
        
    w = interactive(_phasing,
                 ph0=FloatSlider(min=phc0-45,
                                 max=phc0+45,
                                 step=0.001,
                                 value=phc0,
                                 continuous_update=True),
                 ph1=FloatSlider(min=phc1-20,
                                 max=phc1+20,
                                 step=0.001,
                                 value=phc1,
                                 continuous_update=True),
                 pivot=FloatSlider(min=lastcoord[-1].values,
                                   max=lastcoord[0].values,
                                   value=pivot,
                                   continuous_update=True))
    
    return w
    
    


In [ ]:
%matplotlib widget

In [ ]:
w = interact_pk(transf1)
w

In [ ]:
w.children[0].value + dataset1D.meta.phc0[-1]

In [ ]:
x = transf1.plot()

In [ ]:
x.axes.lines[0].set_ydata(0)

In [ ]:
ax = transf1.plot(xlim=[50,-50]).axes
l = ax.lines[0]
p = ax.axvline(0.2, color='r', alpha=0.5)

In [ ]:
p.get_xdata()